# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 1.1.1
- **Date :** 05/11/2023
- **Heure :** 00:36
- **Données :** `ready_Evaporation.csv`
- **Tri :** par défaut
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** KNN + DTW

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Omar/ready_Evaporation.csv", index_col = 1)
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,18.800000,38.4,0.0,13.1,50.000000,22.0,13.0,46.0,20.0,1010.4,...,2009,1,1,38,0.000000,5.890486,5.497787,-31.667778,116.015,5
2009-01-02,22.000000,39.1,0.0,13.2,52.000000,4.0,31.0,38.0,23.0,1008.3,...,2009,1,2,38,0.000000,2.356194,3.926991,-31.667778,116.015,5
2009-01-03,15.900000,35.6,0.0,12.7,43.473477,13.0,28.0,58.0,29.0,1012.1,...,2009,1,3,38,2.896869,9.424778,4.319690,-31.667778,116.015,5
2009-01-04,12.456393,37.0,0.0,13.3,43.000000,24.0,20.0,45.0,19.0,1016.6,...,2009,1,4,38,5.497787,5.497787,5.890486,-31.667778,116.015,5
2009-01-05,21.800000,38.6,0.0,13.2,59.000000,15.0,15.0,35.0,15.0,1014.1,...,2009,1,5,38,0.000000,5.890486,5.105088,-31.667778,116.015,5


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2646 entries, 2009-01-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         2646 non-null   float64
 1   MaxTemp         2646 non-null   float64
 2   Rainfall        2646 non-null   float64
 3   Sunshine        2646 non-null   float64
 4   WindGustSpeed   2646 non-null   float64
 5   WindSpeed9am    2646 non-null   float64
 6   WindSpeed3pm    2646 non-null   float64
 7   Humidity9am     2646 non-null   float64
 8   Humidity3pm     2646 non-null   float64
 9   Pressure9am     2646 non-null   float64
 10  Pressure3pm     2646 non-null   float64
 11  Cloud9am        2646 non-null   float64
 12  Cloud3pm        2646 non-null   float64
 13  Temp9am         2646 non-null   float64
 14  Temp3pm         2646 non-null   float64
 15  RainToday       2646 non-null   int64  
 16  RainTomorrow    2646 non-null   int64  
 17  year            2646 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

`TimeSeriesSplit` est une variation de l'itérateur de validation croisée `KFold` de `scikit-learn`, adaptée aux séries temporelles.

Bien que ces objets soient principalement destinés au découpage des données lors d'un processus de validation croisée, ils peuvent également être appliqués au jeu d'origine pour créer les jeux d'entraînement et de test. D'ailleurs, l'itérateur `train_test_split`, qui est habituellement utilisé à cette étape, n'est autre qu'un _wrapper_ autour de l'itérateur `ShuffleSplit`.

Contrairement aux autres itérateurs, `TimeSeriesSplit` génère des jeux d'entraînement successifs qui sont des surensembles (« supersets ») de ceux qui les précèdent. Traduit dans le cadre temporel, cela indique que les jeux d'entraînement sont systématiquement plus anciens que les jeux de test, ce qui permet de prendre en compte la particularité structurelle des séries temporelles.

![image.png](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

Référence : https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Importation de la classe `KNeighborsTimeSeriesClassifier`

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [10]:
# Instanciation d'un modèle classificateur avec `n_neighbors = 1` et `distance = "dtw"`

clf_knn_ts = KNeighborsTimeSeriesClassifier(n_neighbors = 1, distance = "dtw")

In [11]:
# Récupération des paramètres initiaux

clf_knn_ts.get_params()

{'algorithm': 'brute',
 'distance': 'dtw',
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 1,
 'pass_train_distances': False,
 'weights': 'uniform'}

In [12]:
# Entraînement du modèle

clf_knn_ts.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [13]:
# Récupération des paramètres ajustés

clf_knn_ts.get_fitted_params()

{'classes': array([0, 1]),
 'fit_time': 18,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=1),
 'n_classes': 2,
 'knn_estimator__classes': array([0, 1]),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 2205,
 'knn_estimator__n_samples_fit': 2205,
 'knn_estimator__outputs_2d': False}

In [14]:
# Réalisation des prédictions

y_pred = clf_knn_ts.predict(X_test)

## 5. Évaluation

In [15]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,314,32
1,36,59


In [16]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       346
           1       0.65      0.62      0.63        95

    accuracy                           0.85       441
   macro avg       0.77      0.76      0.77       441
weighted avg       0.84      0.85      0.84       441

